In [ ]:
# hide
from integrators import *

# pyintegrators
> Integrators integrate your information in your Pod. They import data from external services (Gmail, WhatsApp, etc.), enrich data with indexers (face recognition, spam detection, etc.), and execute actions (sending messages, uploading files, etc.).


# Overview

Pyintegrators currently has the following integrators:

In [ ]:
# hide
from IPython.display import Markdown as md
import integrators.integrator_registry
from integrators.data.basic import *
from integrators.imports import *
from nbdev.cli import _test_one

def get_notebook_from_cls(cls):
    path = inspect.getfile(cls)
    f_content = read_file(path)
    file = re.search("(?<=File to edit: )[^\s]*", f_content).group()
    return file[4:] # remove 'nbs/'

txt_passing = '![Build passing](https://gitlab.memri.io/memri/pyintegrators/-/raw/prod/assets/build-passing.svg "Build passing")'
txt_failing = '![Build failing](https://gitlab.memri.io/memri/pyintegrators/-/raw/prod/assets/build-failing.svg "Build failing")'

table = f"""
| Integrator | Description | Tests passing |
|------------|-------------|---------------|
"""

for m in dir(integrators.integrator_registry):
    if "__" not in m:
        cls = getattr(integrators.integrator_registry, m)
        nb = get_notebook_from_cls(cls)
        test_succeeded = _test_one(nb, verbose=False)[0]
        build_txt = txt_passing if test_succeeded else txt_failing
        
        table += f"|`{cls.__name__}`|{cls.__doc__ if cls.__doc__ is not None else ''}| {build_txt}|\n"

testing importers.EmailImporter.ipynb
testing indexers.FaceRecognitionIndexer.ipynb
testing indexers.GeoIndexer.ipynb
testing indexers.NoteListIndexer.ipynb


In [ ]:
# hide_input
md(table)


| Integrator | Description | Tests passing |
|------------|-------------|---------------|
|`EmailImporter`|Imports emails over imap.| ![Build passing](https://gitlab.memri.io/memri/pyintegrators/-/raw/prod/assets/build-passing.svg "Build passing")|
|`FaceRecognitionIndexer`|Recognizes photos from faces.| ![Build passing](https://gitlab.memri.io/memri/pyintegrators/-/raw/prod/assets/build-passing.svg "Build passing")|
|`GeoIndexer`|Adds Countries and Cities to items with a location.| ![Build passing](https://gitlab.memri.io/memri/pyintegrators/-/raw/prod/assets/build-passing.svg "Build passing")|
|`NotesListIndexer`|Extracts lists from notes and categorizes them.| ![Build passing](https://gitlab.memri.io/memri/pyintegrators/-/raw/prod/assets/build-passing.svg "Build passing")|




Every integrator has its own documentation with usage examples. Memri integrators work like plugins, and they are really easy to write yourself. A good place to start would be to check out the `GeoIndexer`, a super simple integrator that shows how you can build your own.

Integrators for Memri have a single repository per language, this repository is the one for Python, but others exist for [Node.js](https://gitlab.memri.io/memri/nodeintegrators) and [Rust](https://gitlab.memri.io/memri/rustintegrators).

# Development

## Install
To install the Python package:
```bash
pip install -e . 
```

## nbdev
This repository makes use of [nbdev](https://github.com/fastai/nbdev), which means all code, documentation and tests are written in [Jupyter Notebooks](https://jupyter.readthedocs.io/en/latest/install/notebook-classic.html). The notebooks are used to generate a Python package, Jekyll documentation and unit tests with nbdev commands.

### nbdev tags
In nbdev we can use cell tags to specify different usage of a cell. See the [nbdev documentation](https://nbdev.fast.ai/) for a all functionalities and tutorials. The most important tags are listed below.

- Notebooks that start their name with an underscore, are ignored by nbdev completely
- Add `#default_exp <packagename>.<modulename>` to the top of your notebook to define the Python module to export to
- Add `#export` to the cells that define functions to include in the Python modules.
- All cells without the `#export` tag, are tests by default
- All cells are included in the documentation, unless you add the keyword `#hide`

### nbdev CLI 
After developing your code in Notebooks, you can use the nbdev CLI:
- `nbdev_build_lib` to convert the Notebooks to the library and tests 
- `nbdev_test_nbs` to run the tests
- `nbdev_build_docs` to generate the docs
- `nbdev_clean_nbs` to clean the Notebooks' metadata to prevent Git conflicts. *In practice you don't have to use this, as explained in the section below.*

## Contributing
As a first step, run `nbdev_install_git_hooks` in the terminal from your project folder. This will set up git hooks which will remove metadata from your notebooks when you commit (with `nbdev_clean_nbs`), greatly reducing the chance you have a conflict.

Before you make a merge request, make sure that you ran `nbdev_build_lib` and check that the tests pass, or GitLab's CI won't pass.

# Deployment

In order to use integrators from one of the front-end applications (e.g. iOS), you need to build a docker image with your integrator. Build it with:
```bash
./tools/build_docker.sh
```

We are planning to allow running a local script instead of a docker image during development. However, if you want to make your integrator available for public use, you will need to build a docker image. We can help you with that if you run into any problems, reach out at our [forum](http://discourse.memri.io).

# Docs
Find the online docs at [pyintegrators.docs.memri.io](https://pyintegrators.docs.memri.io/). The new documentation is automatically generated from the `prod`  branch.


## Render documentation locally
To inspect the documentation locally, you need to [install Jekyll](https://jekyllrb.com/docs/installation/).

Build the documentation with:
```bash
cd docs
gem update --system 
bundle install
```

Serve the documentation with:
```bash
bundle exec jekyll serve
```